#RetrievalQA - advanced techniques

RAG - retrieval augmented generation - is an antidote for contemporary LLMs hallucinations. However, naive RAG is prone to making up facts and not leveraging the knowledge base it has access to. In this notebook we will present techniques for efficient managing available knowledge, advanced retrieval techniqes and reranking capabilities. We base our design on:
 - LangChain RetrievalQA along with Faiss vectore store
 - Llama2-13B as the LLM answer generator
 - Popular SentenceTransformers embeddings

We are going to build solution for two datasets:
1. relatively straightforward books releases dataset
2. intricate cosmology articles dataset

Both of these datasets were scraped from latest articles/data from September 2023 to test the LLM behaviour against data it didn't see.

In [ ]:
!pip install -qU \
  transformers==4.31.0 \
  sentence-transformers==2.2.2 \
  pinecone-client==2.2.2 \
  datasets==2.14.0 \
  accelerate==0.21.0 \
  einops==0.6.1 \
  langchain==0.0.240 \
  xformers==0.0.20 \
  bitsandbytes==0.41.0 \
  faiss-gpu \
  llama_index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 74.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 91.0 MB/

In [ ]:
import os
os.environ['HF_API_KEY'] = ''

#1. Get data about latest books releases

This is a small collections (105 samples) of September novel releases scraped from [goodreads.com](https://www.goodreads.com/book/popular_by_date/2023/9?ref=nav_brws_newrels)

In [ ]:
# load csv data from drive to dataframe and then load it to Dataset dataset
from datasets import Dataset
import pandas as pd


books_df = pd.read_csv("/content/drive/MyDrive/ML/data/book_releases_september.csv")
books_ds = Dataset.from_pandas(books_df)
len(books_ds), books_ds[50]

(105,
 {'id': 'kca://book/amzn1.gr.book.v3.SfQ5kz2CwD7dIfuz',
  'title': 'Witch of Wild Things',
  'description': 'Legend goes that long ago a Flores woman offended the old gods, and their family was cursed as a result. Now, every woman born to the family has a touch of magic.\xa0Sage Flores has been running from her family—and their “gifts”—ever since her younger sister Sky died. Eight years later, Sage reluctantly returns to her hometown. Like slipping into an old, comforting sweater, Sage takes back her job at Cranberry Rose Company and uses her ability to communicate with plants to discover unusual heritage specimens in the surrounding lands.What should be a simple task is complicated by her partner in botany sleuthing: Tennessee Reyes. He broke her heart in high school, and she never fully recovered. Working together is reminding her of all their past tender, genuine moments—and new feelings for this mature sexy man are starting to take root in her heart.With rare plants to find, 

In [ ]:
len(books_df[books_df['description'].isnull()]), len(books_df[books_df['title'].isnull()]), len(books_df[books_df['author'].isnull()])

(2, 0, 0)

In [ ]:
# Replacing NaN values with an empty string
books_df.fillna('', inplace=True)

In [ ]:
len(books_df[books_df['description'].isnull()])

0

#2. Load dataset from NASA ADS

Load a dataset of cosmology articles from last month using NASA ADS database.

In [ ]:
# load csv data from drive to dataframe and then load it to Dataset dataset
from datasets import Dataset
import pandas as pd


nasa_ads_df = pd.read_csv("/content/drive/MyDrive/ML/data/nasa_ads_cosmology_full.csv")
nasa_ads_ds = Dataset.from_pandas(nasa_ads_df)
len(nasa_ads_ds), nasa_ads_ds[180]


(1200,
 {'Unnamed: 0': 180,
  'Title': 'The energy distribution of the first supernovae',
  'Abstract': 'The nature of the first Pop III stars is still a mystery and the energy distribution of the first supernovae is completely unexplored. For the first time we account simultaneously for the unknown initial mass function (IMF), stellar mixing, and energy distribution function (EDF) of Pop III stars in the context of a cosmological model for the formation of a MW-analogue. Our data-calibrated semi-analytic model is based on a N-body simulation and follows the formation and evolution of both Pop III and Pop II/I stars in their proper time-scales. We discover degeneracies between the adopted Pop III unknowns, in the predicted metallicity and carbonicity distribution functions and the fraction of C-enhanced stars. None the less, we are able to provide the first available constraints on the EDF, $dN/dE_\\star \\propto E_{\\star }^{-\\alpha _e}$ with 1 ≤ α<SUB>e</SUB> ≤ 2.5. In addition, the

In [ ]:
nasa_ads_df.head()

,Title,Abstract,Bibcode,Publication Date,Full Text Content (first 10000 words)
0,A CEERS Discovery of an Accreting Supermassive...,We report the discovery of an accreting superm...,2023ApJ...953L..29L,2023-08-00,
1,COSMOS-Web: An Overview of the JWST Cosmic Ori...,"We present the survey design, implementation, ...",2023ApJ...954...31C,2023-09-00,
2,The Eighteenth Data Release of the Sloan Digit...,The eighteenth data release (DR18) of the Sloa...,2023ApJS..267...44A,2023-08-00,
3,The FLAMINGO project: cosmological hydrodynami...,We introduce the Virgo Consortium's FLAMINGO s...,2023MNRAS.tmp.2384S,2023-08-00,
4,Hidden Little Monsters: Spectroscopic Identifi...,We report on the discovery of two low-luminosi...,2023ApJ...954L...4K,2023-09-00,


All recores have bibcode and each is unique:

In [ ]:
len(nasa_ads_df[nasa_ads_df['Bibcode'].isnull()]), nasa_ads_df['Bibcode'].nunique()

(0, 1200)

#3. Get LLama2-13B and Embeddings

##3.1. Llama2-13B

We load Llama2-13B, fitting it on a single 15GB RAM with the help of bitsandbytes quantization.

In [ ]:
%%time
from torch import cuda, bfloat16
import transformers

model_name = "meta-llama/Llama-2-13b-chat-hf"
# set quantization configuration using bitsandbytes lib
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)


hf_auth = os.environ.get('HF_API_KEY')
model_config = transformers.AutoConfig.from_pretrained(
    model_name,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)
model.eval()


config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

CPU times: user 43.9 s, sys: 52.5 s, total: 1min 36s
Wall time: 4min 1s


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 5120, padding_idx=0)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear4bit(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )


In [ ]:
%%time
import torch
model_name = "meta-llama/Llama-2-13b-chat-hf"
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_name,
    use_auth_token=hf_auth
)


generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    temperature=0.0,
    max_new_tokens=256,  # number of tokens to generate in the output
    repetition_penalty=1.1
)



tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

CPU times: user 947 ms, sys: 129 ms, total: 1.08 s
Wall time: 2.95 s


## 3.2. Get embeddings

Let's select some decent embeddings model based on [HF's leaderboard](https://huggingface.co/spaces/mteb/leaderboard).

In [ ]:

from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embed_model_id = 'intfloat/e5-base-v2' # at the time #14 on HF's MTEB leaderboard, dim=768

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32}
)

In [ ]:
books_df.head()

,id,title,description,webUrl,author,rank
0,kca://book/amzn1.gr.book.v3.CI518ofIFidEXILu,Things We Left Behind,There was only one woman who could set me free...,https://www.goodreads.com/book/show/116536542-...,Lucy Score,1
1,kca://book/amzn1.gr.book.v3.laK_97rF9qQjQGbP,The Long Game,A disgraced soccer exec reluctantly enlists th...,https://www.goodreads.com/book/show/101144869-...,Elena Armas,2
2,kca://book/amzn1.gr.book.v3.AbK983RbgzqySZEv,Bright Young Women,An extraordinary novel inspired by the real-li...,https://www.goodreads.com/book/show/101124639-...,Jessica Knoll,3
3,kca://book/amzn1.gr.book.v3.WMf4mFalogDRVJoY,The Fragile Threads of Power,From the #1 New York Times bestselling author ...,https://www.goodreads.com/book/show/111673828-...,V.E. Schwab,4
4,kca://book/amzn1.gr.book.v3.fDu8MXBOGcUfyd89,Rouge,From the critically acclaimed author of Bunny ...,https://www.goodreads.com/book/show/101160689-...,Mona Awad,5


#4. Basic LangChain setup

We initialize two modules from Langchain: the Hugginface llm pipeline and the RetrievalQA chain and compare their outputs against a set of questions based on the book releases from September from goodreads.

We will be using RetrievalQA in the following setup:
- text split with LC's recursive text splitter, chunk sizes of around 500 tokens,  overlap of 70 tokens
- FAISS as vector database - in-memory memory data vector store based on Meta's optimized library Faiss for efficient similarity search
- Llama-2 13B to generate the final output from RAG

In [ ]:
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import CSVLoader, DataFrameLoader
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA

llm = HuggingFacePipeline(pipeline=generate_text)

def combineBooks(row):
    return f"Author: {row['author']}, Title: '{row['title']}', Description: {row['description']}"

def combineCosmo(row):
    return f"Title: '{row['Title']}', Abstract: {row['Abstract']}"


# books_df['combined'] =  "Author: " + books_df['author'] + ", Title: " + books_df['title'] + ", Description: " + books_df['description']  + " Id: " + books_df['id']
books_df['combined'] = books_df.apply(lambda x: combineBooks(x), axis=1)
booksLoader = DataFrameLoader(books_df[["combined", "id"]], page_content_column="combined")
books_documents = booksLoader.load()

nasa_ads_df['combined'] = nasa_ads_df.apply(lambda x: combineCosmo(x), axis=1)
cosmoLoader = DataFrameLoader(nasa_ads_df[["combined"]], page_content_column="combined")
cosmo_documents = cosmoLoader.load()

def splitTexts(documents, chunk_size, chunk_overlap, length_function = len):
    text_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n", " ", ""],
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=length_function,
    )

    texts = text_splitter.split_documents(documents)
    return texts

# Text splitting
books_texts = splitTexts(books_documents, chunk_size=500, chunk_overlap=70)
cosmo_texts = splitTexts(cosmo_documents, chunk_size=400, chunk_overlap=70)

def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))

In [ ]:
texts = cosmo_texts

dbFaiss = FAISS.from_documents(texts, embed_model)
retrieverFaiss = FAISS.from_documents(texts, embed_model).as_retriever()

rag = RetrievalQA.from_chain_type(
    llm=llm, chain_type='stuff', verbose=True,
    retriever=retrieverFaiss
)


In [ ]:
books_texts[:5]

[Document(page_content="Author: Lucy Score, Title: 'Things We Left Behind', Description: There was only one woman who could set me free. But I would rather set myself on fire than ask Sloane Walton for anything.Lucian Rollins is a lean, mean vengeance-seeking mogul. On a quest to erase his father’s mark on the family name, he spends every waking minute pulling strings and building an indestructible empire. The more money and power he amasses, the safer he is from threats.Except when it comes to the feisty small-town", metadata={'id': 'kca://book/amzn1.gr.book.v3.CI518ofIFidEXILu'}),
 Document(page_content='he is from threats.Except when it comes to the feisty small-town librarian that keeps him up at night…Sloane Walton is a spitfire determined to carry on her father’s quest for justice. She’ll do that just as soon as she figures out exactly what the man she hates did to—or for—her family. Bonded by an old, dark secret from the past and the dislike they now share for each other, Sloane

In [ ]:
cosmo_texts[:5]

[Document(page_content="Title: 'A CEERS Discovery of an Accreting Supermassive Black Hole 570 Myr after the Big Bang: Identifying a Progenitor of Massive z &gt; 6 Quasars', Abstract: We report the discovery of an accreting supermassive black hole at z = 8.679. This galaxy, denoted here as CEERS_1019, was previously discovered as a Lyα-break galaxy by Hubble with a Lyα redshift from Keck. As part of the Cosmic Evolution", metadata={}),
 Document(page_content='Hubble with a Lyα redshift from Keck. As part of the Cosmic Evolution Early Release Science (CEERS) survey, we have observed this source with JWST/NIRSpec, MIRI, NIRCam, and NIRCam/WFSS and uncovered a plethora of emission lines. The Hβ line is best fit by a narrow plus a broad component, where the latter is measured at 2.5σ with an FWHM ~1200 km s<SUP>-1</SUP>. We conclude this originates in the', metadata={}),
 Document(page_content='an FWHM ~1200 km s<SUP>-1</SUP>. We conclude this originates in the broadline region of an active

#6. Testing the basic setup
Compare pure LLM answers and RetrievalQA answers to following questions:

##6.1. Book releases questions



In [ ]:
llm("What is the story of Lucian and Sloane in 'Things We Left Behind'?")

'\n\nAnswer: In "Things We Left Behind," Lucian and Sloane are a couple who have been together for several years. They have a comfortable life together, but they both feel like something is missing. They decide to take a trip to a remote island to try and find what they\'re looking for. During their time on the island, they encounter various challenges and obstacles, including a mysterious illness that affects them both. As they work together to overcome these challenges, they begin to realize that what they\'ve been searching for all along is each other. The story explores themes of love, loss, and the power of human connection.'

In [ ]:
rag("What is the story of Lucian and Sloane in 'Things We Left Behind'?")



> Entering new RetrievalQA chain...

> Finished chain.


{'query': "What is the story of Lucian and Sloane in 'Things We Left Behind'?",
 'result': " Based on the provided text, there is no mention of a specific story about Lucian and Sloane in 'Things We Left Behind'. The text only describes the character of Lucian Rollins as a lean, mean vengeance-seeking mogul who is on a quest to erase his father's mark on the family name, and mentions that he has cut Sloane Walton out of his life twice before. It does not provide any information about their relationship or interactions. Therefore, I cannot answer the question based on the given context."}

Let's check similarity search results for Faiss (returned with scores. Score is  L2 distance, therfore, a lower score is higher similarity).

In [ ]:
dbFaiss.similarity_search_with_score(
    "What is the story of Lucian and Sloane in 'Things We Left Behind'?",  # the search query
    k=4
)

[(Document(page_content="Author: Lucy Score, Title: 'Things We Left Behind', Description: There was only one woman who could set me free. But I would rather set myself on fire than ask Sloane Walton for anything.Lucian Rollins is a lean, mean vengeance-seeking mogul. On a quest to erase his father’s mark on the family name, he spends every waking minute pulling strings and building an indestructible empire. The more money and power he amasses, the safer he is from threats.Except when it comes to the feisty small-town", metadata={'id': 'kca://book/amzn1.gr.book.v3.xbX1ZuO1jHnpidIr'}),
  0.26563796),
 (Document(page_content="Author: Lucy Score, Title: 'Things We Left Behind', Description: There was only one woman who could set me free. But I would rather set myself on fire than ask Sloane Walton for anything.Lucian Rollins is a lean, mean vengeance-seeking mogul. On a quest to erase his father’s mark on the family name, he spends every waking minute pulling strings and building an indest

In [ ]:
rag( "What is the plot of Lucy Score's novel?")



> Entering new RetrievalQA chain...

> Finished chain.


{'query': "What is the plot of Lucy Score's novel?",
 'result': ' Based on the provided descriptions, the plot of Lucy Score\'s novel "Things We Left Behind" appears to revolve around the main character, Lucian Rollins, and his quest for revenge against his father\'s legacy, as well as his complicated relationship with the feisty small-town woman, Sloane Walton.'}

In [ ]:
dbFaiss.similarity_search_with_score(
    "What is the plot of Lucy Score's novel?",  # the search query
    k=3
)

[(Document(page_content="Author: Lucy Score, Title: 'Things We Left Behind', Description: There was only one woman who could set me free. But I would rather set myself on fire than ask Sloane Walton for anything.Lucian Rollins is a lean, mean vengeance-seeking mogul. On a quest to erase his father’s mark on the family name, he spends every waking minute pulling strings and building an indestructible empire. The more money and power he amasses, the safer he is from threats.Except when it comes to the feisty small-town", metadata={'id': 'kca://book/amzn1.gr.book.v3.xbX1ZuO1jHnpidIr'}),
  0.37934643),
 (Document(page_content="Author: Lucy Score, Title: 'Things We Left Behind', Description: There was only one woman who could set me free. But I would rather set myself on fire than ask Sloane Walton for anything.Lucian Rollins is a lean, mean vengeance-seeking mogul. On a quest to erase his father’s mark on the family name, he spends every waking minute pulling strings and building an indest

In [ ]:
rag( "What is the plot of 'Things We Left Behind'?")



> Entering new RetrievalQA chain...

> Finished chain.


{'query': "What is the plot of 'Things We Left Behind'?",
 'result': " Based on the description provided, 'Things We Left Behind' appears to be a romantic mystery novel that revolves around the story of Lucian Rollins, a wealthy businessman seeking revenge against his father's legacy, and his complicated relationship with Sloane Walton, the woman who can set him free. The book seems to explore themes of deception, power, and the consequences of one's actions."}

In [ ]:
dbFaiss.similarity_search_with_score(
    "What is the plot of 'Things We Left Behind'?",
    k=3
)

[(Document(page_content="Author: Lucy Score, Title: 'Things We Left Behind', Description: There was only one woman who could set me free. But I would rather set myself on fire than ask Sloane Walton for anything.Lucian Rollins is a lean, mean vengeance-seeking mogul. On a quest to erase his father’s mark on the family name, he spends every waking minute pulling strings and building an indestructible empire. The more money and power he amasses, the safer he is from threats.Except when it comes to the feisty small-town", metadata={'id': 'kca://book/amzn1.gr.book.v3.xbX1ZuO1jHnpidIr'}),
  0.3818617),
 (Document(page_content="Author: Lucy Score, Title: 'Things We Left Behind', Description: There was only one woman who could set me free. But I would rather set myself on fire than ask Sloane Walton for anything.Lucian Rollins is a lean, mean vengeance-seeking mogul. On a quest to erase his father’s mark on the family name, he spends every waking minute pulling strings and building an indestr

LLM's answer is completely made up and not true to facts.

We can see that if the book was matched positively against the appropriate record the proper answer is returned by the retriever chain.


In [ ]:
rag("What is the plot of 'Things We Left Behind'?")




> Entering new RetrievalQA chain...

> Finished chain.


{'query': "What is the plot of 'Things We Left Behind'?",
 'result': " Based on the description provided, 'Things We Left Behind' appears to be a romantic mystery novel that revolves around the story of Lucian Rollins, a wealthy businessman seeking revenge against his father's legacy, and his complicated relationship with Sloane Walton, the woman who can set him free. The book seems to explore themes of deception, power, and the consequences of one's actions."}

In [ ]:
llm( "What are the latest adventures of Holly Gibney and what case is she solving in King's latest novel and what's it called?")


'\n\nAnswer: The latest adventures of Holly Gibney can be found in the novel "The Outsider" by Stephen King, which was published in 2018. In this book, Holly is a key player in solving a gruesome murder that takes place in a small town in Oklahoma. The case centers around the death of a young boy who is found dead in a cave, with no signs of any suspects or motives. Holly uses her unique abilities as an autistic savant to help unravel the mystery and bring the perpetrator to justice.'

In [ ]:
rag( "What are the latest adventures of Holly Gibney and what case is she solving in King's latest novel and what's it called?")



> Entering new RetrievalQA chain...

> Finished chain.


{'query': "What are the latest adventures of Holly Gibney and what case is she solving in King's latest novel and what's it called?",
 'result': " Holly Gibney, the protagonist of King's latest novel, is on her own and up against a pair of unimaginably depraved and brilliantly disguised adversaries as she tries to solve the gruesome truth behind multiple disappearances in a midwestern town."}

In [ ]:
dbFaiss.similarity_search_with_score(
    "What are the latest adventures of Holly Gibney and what case is she solving in King's latest novel and what's it called?",
    k=3
)

[(Document(page_content="Author: Stephen King, Title: 'Holly', Description: Holly Gibney, one of Stephen King’s most compelling and ingeniously resourceful characters, returns in this thrilling novel to solve the gruesome truth behind multiple disappearances in a midwestern town.“Sometimes the universe throws you a rope.” — BILL HODGESStephen King’s Holly marks the triumphant return of beloved King character Holly Gibney. Readers have witnessed Holly’s gradual transformation from a shy (but also brave and ethical)", metadata={'id': 'kca://book/amzn1.gr.book.v3.Xdyc7bcQBYUrWE8s'}),
  0.2978371),
 (Document(page_content="Author: Stephen King, Title: 'Holly', Description: Holly Gibner, één van Stephen Kings meest meeslepende en vindingrijke personages, keert in deze bloedstollende thriller terug om de gruwelijke waarheid achter een reeks verdwijningen in een klein stadje te achterhalen.Stephen Kings Holly betekent de triomfantelijke terugkeer van het geliefde personage Holly Gibner. Lezer

Pure LLM produced wrong answer because his parametric knowledge doesnt contain information about the latest S. King book "Holly".

 RAG finds the right answer but LLM doesn't mention the title in its answer.


In [ ]:
llm("What is the latest installment in the Serpent & Dove NYT series about and what's called and what happens to Célie?")


'\n\nThe latest installment of the Serpent & Dove series by The New York Times is called "Célie\'s Journey" and it follows the story of Célie, a young woman who was introduced in the previous installment as a new member of the group. In this installment, Célie embarks on a journey to discover her true identity and purpose, and she encounters many challenges and obstacles along the way.\n\nOne of the main plot points in "Célie\'s Journey" is that Célie discovers that she has the ability to control the elements, specifically water. She learns how to harness her powers and use them for good, but she also faces the temptation of using her powers for personal gain. This internal conflict is a central theme of the installment and adds depth to Célie\'s character.\n\nAnother important aspect of "Célie\'s Journey" is the development of the relationships between the characters. Célie forms close bonds with some of the other members of the group, particularly with a character named Lysander, who

In [ ]:
rag("What is the latest installment in the Serpent & Dove NYT series about and what happens to Célie?")



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'What is the latest installment in the Serpent & Dove NYT series about and what happens to Célie?',
 'result': ' The latest installment is called "The Scarlet Veil" and it follows Célie as she takes her sacred vows and joins the ranks of the Chasseurs as their first huntswoman. She faces new challenges and old demons from her past while trying to protect Belterra from a new evil rising.'}

Here both LLM and RAG gave wrong made up titles.  It's actually called 'The secret veil". The rest of the answer is right in case of RAG.

Below slightly more general question about the same resulted in query not being matched agaist the correct vectors in the db at all.

In [ ]:
dbFaiss.similarity_search_with_score(
    "What is the latest installment in the Serpent & Dove NYT series about and what's it called?",  # the search query
    k=3
)

[(Document(page_content="Author: Shelby Mahurin, Title: 'The Scarlet Veil', Description: A dark and thrilling vampire romance set in the world of the New York Times bestselling Serpent &amp; Dove series.Six months have passed since Célie took her sacred vows and joined the ranks of the Chasseurs as their first huntswoman. With her fiancé, Jean Luc, as captain, she is determined to find her foothold in her new role and help protect Belterra. But whispers from her past still haunt her, and a new evil is rising—one that", metadata={'id': 'kca://book/amzn1.gr.book.v3.SLNfI1TaGG52CXO1'}),
  0.39994842),
 (Document(page_content='firmly on their tail, has their luck finally run out? And who will be the last devil to die?', metadata={'id': 'kca://book/amzn1.gr.book.v3.28cOdXrYaG19ApBY'}),
  0.40393052),
 (Document(page_content='an unforgettable novel about secrets and fates that asks the timeless how do we live on, even after we’re gone?', metadata={'id': 'kca://book/amzn1.gr.book.v3.uzHBgqjf0

In [ ]:
llm( "What is the sequel to 'Foul Lady Fortune' and what is it about?")


'\n\nAnswer: The sequel to "Foul Lady Fortune" is called "The Sweetest Kind of Cruelty". It continues the story of Arsinoe and her friends as they navigate the dangerous world of the game. The book explores themes of loyalty, power, and the cost of ambition as the characters face new challenges and obstacles in their quest for victory.'

In [ ]:
rag( "What is the sequel to 'Foul Lady Fortune' and what is it about?")



> Entering new RetrievalQA chain...

> Finished chain.


{'query': "What is the sequel to 'Foul Lady Fortune' and what is it about?",
 'result': " The sequel to 'Foul Lady Fortune' is titled 'Our Violent Ends'. It continues the story of Rosalind Lang, an immortal assassin in 1930s Shanghai, as she races to save her country and her love while facing the ever-nearing threat of a Japanese invasion."}

In [ ]:
dbFaiss.similarity_search_with_score(
    "What is the sequel to 'Foul Lady Fortune' and what is it about?",  # the search query
    k=3
)

[(Document(page_content="Author: Chloe Gong, Title: 'Foul Heart Huntsman', Description: From the #1 New York Times bestselling author of These Violent Delights and Our Violent Ends comes the second book in the captivating Foul Lady Fortune duology following an immortal assassin in 1930s Shanghai as she races to save her country and her love.Winter is drawing thick in 1932 Shanghai, as is the ever-nearing threat of a Japanese invasion.Rosalind Lang has suffered the worst possible fate for a national spy: she’s been", metadata={'id': 'kca://book/amzn1.gr.book.v3.cmX8HeCZaDYgA_Vk'}),
  0.39501286),
 (Document(page_content='an unforgettable novel about secrets and fates that asks the timeless how do we live on, even after we’re gone?', metadata={'id': 'kca://book/amzn1.gr.book.v3.uzHBgqjf0cRXilp8'}),
  0.42079186),
 (Document(page_content='Author: Kathryn Purdie, Title: \'The Forest Grimm\', Description: Where fairy tales come to life with dark, deadly twists..."Tell me again, Grandmère, t

LLM confabulated about the title and the plot of the story. RAG also mistaken title for the title for another book from that same author that it found in the description of the novel, but the rest of the plot is well summarized.



In [ ]:
llm( "Give me a short synopsis of events of 'The Unfortunate Side Effects of Heartbreak and Magic' novel by B.Randall")


'.\n\nSure! Here\'s a short synopsis of the events in "The Unfortunate Side Effects of Heartbreak and Magic" by B. Randall:\n\nAfter her boyfriend dumps her, Ember finds solace in her magic abilities. However, she soon discovers that her heartbreak has unleashed a powerful and unpredictable magic within her. As she struggles to control her newfound powers, Ember must also navigate her complicated relationships with her best friend, her ex-boyfriend, and a mysterious new love interest. Along the way, she learns that the line between magic and heartbreak is thin, and that true love can be found in the most unexpected places.'

In [ ]:
rag( "Give me a short synopsis of events of 'The Unfortunate Side Effects of Heartbreak and Magic' novel by B.Randall")



> Entering new RetrievalQA chain...

> Finished chain.


{'query': "Give me a short synopsis of events of 'The Unfortunate Side Effects of Heartbreak and Magic' novel by B.Randall",
 'result': " Sure! Here's a brief summary of the plot of 'The Unfortunate Side Effects of Heartbreak and Magic':\n\nSadie Revelare, a young woman with magical abilities, must navigate her family's history of heartbreak and the consequences it brings. As her grandmother's health declines and her own relationships falter, Sadie must confront the last of her heartbreaks and decide whether love is more important than her magic. Along the way, she must also contend with a mysterious force threatening her town and learn to trust those around her in order to save everything she holds dear."}

Here Llama hallucinates again, and retriever does good job. 3:0 to RAG.

In [ ]:
llm( "What's the latest novel by Mona Awad?")


'\n\nAnswer: The latest novel by Mona Awad is "Bewilderness".'

In [ ]:
rag( "What's the latest novel by Mona Awad?")



> Entering new RetrievalQA chain...

> Finished chain.


{'query': "What's the latest novel by Mona Awad?",
 'result': ' Based on the information provided, the latest novel by Mona Awad is "Rouge".'}

In [ ]:
dbFaiss.similarity_search_with_score(
    "What's the latest novel by Mona Awad?",  # the search query

    k=3  # returns top 3 most relevant articles
)

[(Document(page_content="Author: Mona Awad, Title: 'Rouge', Description: From the critically acclaimed author of Bunny comes a horror-tinted, gothic fairy tale about a lonely dress shop clerk whose mother’s unexpected death sends her down a treacherous path in pursuit of youth and beauty. Can she escape her mother’s fate—and find a connection that is more than skin deep?For as long as she can remember, Belle has been insidiously obsessed with her skin and skincare videos. When her estranged mother Noelle mysteriously", metadata={'id': 'kca://book/amzn1.gr.book.v3.fDu8MXBOGcUfyd89'}),
  0.32441324),
 (Document(page_content='an unforgettable novel about secrets and fates that asks the timeless how do we live on, even after we’re gone?', metadata={'id': 'kca://book/amzn1.gr.book.v3.uzHBgqjf0cRXilp8'}),
  0.42721808),
 (Document(page_content="Author: Etaf Rum, Title: 'Evil Eye', Description: The acclaimed New York Times bestselling author of A Woman Is No Man returns with a striking explor

Here pure LLM was wrong while RAGs answer while correct is pretty brief.



In [ ]:
llm("What does Mona Awad say about beauty industry in her latest book?")

'\nMona Awad\'s latest book, "All the Wild Hungers", explores themes of beauty, identity, and power through the lens of a group of women who are struggling to find their place in society. In the book, Awad challenges traditional notions of beauty and femininity, arguing that these concepts are socially constructed and can be damaging to individuals and society as a whole.\n\nOne of the main characters in the book, Lily, is a young woman who is obsessed with her appearance and feels pressure to conform to societal beauty standards. Through Lily\'s journey, Awad highlights the ways in which the beauty industry can be harmful and oppressive, perpetuating unrealistic beauty ideals and reinforcing damaging gender stereotypes.\n\nAt the same time, Awad also celebrates the beauty and strength of women in all their forms, suggesting that true beauty comes from within and cannot be reduced to physical appearances. Through her writing, Awad encourages readers to embrace their unique qualities an

In [ ]:
rag("What does Mona Awad say about beauty industry in her latest book?")



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'What does Mona Awad say about beauty industry in her latest book?',
 'result': ' Based on the description provided, it appears that Mona Awad\'s latest book, "Rouge," explores the darker aspects of the beauty industry and the impact it can have on individuals, particularly women. The book touches on themes such as obsession, grief, and the complex relationships between mothers and daughters. It also uses elements of horror and black humor to examine the cult-like nature of the beauty industry and the dangers of internalizing its pitiless gaze.'}

In [ ]:
dbFaiss.similarity_search_with_score(
    "What does Mona Awad say about beauty industry in her latest book?",  # the search query
    k=2
)

[(Document(page_content="Author: Mona Awad, Title: 'Rouge', Description: From the critically acclaimed author of Bunny comes a horror-tinted, gothic fairy tale about a lonely dress shop clerk whose mother’s unexpected death sends her down a treacherous path in pursuit of youth and beauty. Can she escape her mother’s fate—and find a connection that is more than skin deep?For as long as she can remember, Belle has been insidiously obsessed with her skin and skincare videos. When her estranged mother Noelle mysteriously", metadata={'id': 'kca://book/amzn1.gr.book.v3.fDu8MXBOGcUfyd89'}),
  0.32243598),
 (Document(page_content="an industry that seems determined to keep pushing them together, can Blake hold on to her grudge? Or will she finally warm up to Gabriel's charm?", metadata={'id': 'kca://book/amzn1.gr.book.v3.1kpTSeXV5Lim2cs1'}),
  0.42276725)]

**Summary:**

Overall, even the naive RAG setup: Llama + vector store + embeddings setup does much better job of finding correct answers. Vector store component seems to work very well, and the LLM sometimes gets confused about titles, but otherwise produce true information.

Big improvement overall!

##6.2 Cosmology questions

Now we pick some test questions based on NASA cosmology corpus of articles from last month (August-September). This dataset contains much more intricate, jargon-packed langugae.




In [ ]:
llm('Why are young massive planets easier to study their atmospheres?')

'\n\nYoung massive planets are easier to study their atmospheres because they are typically brighter and more massive than older, smaller planets. This makes them easier to detect and observe with telescopes, allowing scientists to gather more data about their atmospheres. Additionally, young massive planets are often still in the process of forming, which means that their atmospheres are still being shaped by ongoing planetary accretion and differentiation. This makes them a unique window into the early stages of planetary formation and evolution.\n\n\n\n'

In [ ]:
rag('Why are young massive planets easier to study their atmospheres?')



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Why are young massive planets easier to study their atmospheres?',
 'result': ' Young massive planets are easier to study because they are more luminous, allowing for better detection and analysis of their atmospheres through spectroscopy. Additionally, their close proximity to the host star means that their atmospheres are more extended and easier to observe.'}

In [ ]:
dbFaiss.similarity_search_with_score(
    'Why are young massive planets easier to study their atmospheres?',  # the search query

    k=3  # returns top 2 most relevant articles
)

[(Document(page_content="Title: 'CO, H$_2$O, and CH$_4$ in the Dusty Atmosphere of a $\\le$5 Myr-old Exoplanet', Abstract: Very young massive planets are sufficiently luminous by their internal heat of formation to permit detailed studies, including spectroscopy of their atmospheres with large telescopes at sufficient resolution ($\\lambda / \\Delta \\lambda \\gtrsim 1000$) to identify major constituents to inform models of", metadata={}),
  0.26380932),
 (Document(page_content="Title: 'Searching for Novel Chemistry in Exoplanetary Atmospheres using Machine Learning for Anomaly Detection', Abstract: The next generation of telescopes will yield a substantial increase in the availability of high-resolution spectroscopic data for thousands of exoplanets. The sheer volume of data and number of planets to be analyzed greatly motivate the development of new, fast and efficient", metadata={}),
  0.29606882),
 (Document(page_content="Title: 'Interior-atmosphere modelling to assess the observab

Here both models give correct answer but the pure Llama-2 elaborates on the topic in more comprehensible way.


In [ ]:
llm('What are the latest TESS observations of magnetic hot stars?')

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


'\n\nThe Transiting Exoplanet Survey Satellite (TESS) has made numerous observations of magnetic hot stars, providing valuable insights into their properties and behavior. Here are some of the latest TESS observations of magnetic hot stars:\n\n1. Magnetic field strengths: TESS has measured the magnetic field strengths of several magnetic hot stars, including the star HD 47368, which has a surface magnetic field of about 2.5 kG (Koch et al. 2020). These measurements have helped to constrain models of magnetic hot star evolution and have provided new insights into the role of magnetic fields in shaping the properties of these stars.\n2. Rotation periods: TESS has determined the rotation periods of several magnetic hot stars, including the star HD 191611, which rotates every 1.3 days (Huber et al. 2020). These measurements have helped to constrain models of magnetic hot star evolution and have provided new insights into the relationship between magnetic fields and rotation in these stars.

In [ ]:
rag('What are the latest TESS observations of magnetic hot stars?')



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'What are the latest TESS observations of magnetic hot stars?',
 'result': ' The latest TESS observations of magnetic hot stars include the photometric variability and stochastic low frequency (SLF) analysis of the LAMOST sample of magnetic chemically peculiar stars, as well as the variability catalog of stars observed during the TESS Prime Mission. These observations provide valuable information about the internal effects of surface magnetic fields in these stars and offer insights into their rotational and pulsational properties.'}

In [ ]:
dbFaiss.similarity_search_with_score(
    'What are the latest TESS observations of magnetic hot stars?',  # the search query
    k=4
)

[(Document(page_content="Title: 'Variability of magnetic hot stars from the TESS observations', Abstract: Magnetic hot stars refer to the stars, which effective temperatures approximately in the range from 7,000 to 50,000 K, and with large-scale globally organized magnetic fields. These magnetic fields exhibit strengths ranging from tens of Gauss to tens of kilo-Gauss. They are key in understanding the effects caused by", metadata={}),
  0.24958429),
 (Document(page_content='of kilo-Gauss. They are key in understanding the effects caused by magnetic fields in the stellar evolution. However, there are only three magnetic hot stars studied via a combination of spectropolarimetric and asteroseismic modeling. Combined with $Transiting\\;Exoplanet\\;Survey\\;Satellite\\;(TESS)$ 1-56 sectors data sets, we provided a photometric variability and stochastic low frequency (SLF)', metadata={}),
  0.2839116),
 (Document(page_content="Title: 'Photometric variability of the LAMOST sample of magnetic

This question was pretty general and both models answered differently (Llama based on its parametric knowledge only and retriever based on the paper it found relevant).

In [ ]:
llm('What is IMF function and what is it used for?')

"\n\nThe IMF (International Monetary Fund) is an international organization that provides financial assistance to countries in economic distress. The IMF's primary function is to provide loans to member countries facing economic difficulties, with the goal of helping them restore stability to their economies and achieve long-term growth.\n\nThe IMF also provides policy advice and technical assistance to its member countries, aimed at improving their economic management and promoting economic stability. In addition, the IMF monitors the economic health of its member countries and works to promote global economic cooperation and stability.\n\nThe IMF's functions include:\n\n1. Providing loans to member countries: The IMF offers loans to member countries facing economic difficulties, with the goal of helping them address short-term financing needs and restore stability to their economies.\n2. Providing policy advice and technical assistance: The IMF offers advice and assistance on economi

In [ ]:
rag('What is IMF function and what is it used for?')



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'What is IMF function and what is it used for?',
 'result': ' The IMF stands for Initial Mass Function, which describes how many stars form with different masses when a cloud of gas collapses to form stars. The IMF is typically expressed as a power law relationship between the number of stars and their masses, with more massive stars being less common. The IMF is used to predict the number of stars that will form in a given region of space based on the amount of gas available and other factors such as the strength of the radiation field and the presence of dust.\n\nContext: The text discusses the use of the IMF to predict the number of stars that will form in a given region of space, and how the IMF can be affected by factors such as the strength of the radiation field and the presence of dust. The text also mentions that the IMF should be sensitive to dust properties/abundance in weak radiation fields, but not in strong radiation fields.'}

In [ ]:
dbFaiss.similarity_search_with_score(
    'What is IMF function and what is it used for',  # the search query
    k=3  # returns top 3 most relevant articles
)

[(Document(page_content='If the radiation field is weak or dust opacity is small, $M_{\\rm J}^{\\rm min}$ is insensitive to dust properties/abundance ($\\sim A_{\\rm d}^{-\\frac{1}{11}}-A_{\\rm d}^{-\\frac{1}{15}}$), but if the radiation field is strong and dust is abundant it scales $\\propto A_{\\rm d}^{1/3}$. This could make the IMF less bottom-heavy in dust-rich and/or radiation-dense environments, e.g. galactic centers,', metadata={}),
  0.3419421),
 (Document(page_content='technique, spectral energy distributions are modeled assuming a stellar initial mass function (IMF) that depends on the temperature of gas in star-forming regions. The method is applied to the largest current sample of high-quality panchromatic photometry, the COSMOS2020 catalog, that allows for testing this assumption. It is found that the galaxies exhibit a continuum of IMF and gas', metadata={}),
  0.3549576),
 (Document(page_content="by other means. 2. Update of mama/src/libs/gf2/foldingy.f New SuN excitati

Here IMF in question is actually Initial Mass Function, based on the articles set provided. It's pretty obscure piece of information contained in one of the articles, but RAG finds it without problems. Llama refered to more pupular IMF abbreviation in its answer.

In [ ]:
dbFaiss.similarity_search_with_score(
    'What is SNCosmo for python?',  # the search query
    k=2
)

[(Document(page_content="Title: 'SNCosmo', Abstract: Python library for supernova cosmology", metadata={}),
  0.22639911),
 (Document(page_content="Title: 'Python script for stochastic floodplain modeling information system (SFMIS) framework', Abstract: Python script co-built with the GPT-3 tool from scratch for the SFMIS framework. The script handles and treats all the necessary data for: the creation of stochastic samples for 2D flood models by Latin hypercube sampling (LHS); the automated execution and modification of all the hydraulic", metadata={}),
  0.37035748)]

In [ ]:
dbFaiss.similarity_search_with_score(
    'What is SNCosmo ?',  # the search query
    k=3
)

[(Document(page_content="Title: 'SNCosmo', Abstract: Python library for supernova cosmology", metadata={}),
  0.29331285),
 (Document(page_content="Title: 'A detailed look at the thermal and nonthermal X-ray emission from the Vela supernova remnant with SRG/eROSITA', Abstract: Context. The Vela supernova remnant (SNR) is one of the most nearby and extended objects in the X-ray sky. It constitutes a unique laboratory for studying the thermal and nonthermal X-ray emission from an evolved SNR and its central plerion at an unprecedented level of", metadata={}),
  0.39998147),
 (Document(page_content="Title: 'SN 2020bio: A Double-peaked, H-poor Type IIb Supernova with Evidence of Circumstellar Interaction', Abstract: We present photometric and spectroscopic observations of SN 2020bio, a double-peaked Type IIb supernova (SN) discovered within a day of explosion, primarily obtained by Las Cumbres Observatory and Swift. SN 2020bio displays a rapid and long-lasting initial decline throughout th

In [ ]:
llm('What is SNCosmo?')

'\n\nSNCosmo is a software tool that helps astronomers and researchers analyze and visualize large datasets from telescopes and other sources. It provides a range of features and functionalities to help users process, visualize, and understand their data. Some of the key features of SNCosmo include:\n\n1. Data processing: SNCosmo can handle a wide range of astronomical data formats, including FITS, IRAF, and PyRAF. It provides tools for cleaning, calibrating, and reducing the data, as well as for performing basic analysis tasks such as sky subtraction and flux calibration.\n2. Visualization: SNCosmo includes a range of visualization tools, including 2D and 3D plotting, image display, and movie generation. Users can customize the appearance of their plots and images, and can also create interactive visualizations using Python scripting.\n3. Analysis: SNCosmo provides a range of analysis tools for studying the properties of astronomical objects and phenomena. These tools include spectros

In [ ]:
rag('What is SNCosmo?')



> Entering new RetrievalQA chain...


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.


{'query': 'What is SNCosmo?',
 'result': ' SNCosmo is a Python library for supernova cosmology.'}

General question: What is SNCosmo yields just brief information from the article base. LLM by itself gives better answer to the question, while RAG keeps it very close to information retrieved.


In [ ]:
dbFaiss.similarity_search_with_score(
    'How are solar winds connected to picoflares?',  # the search query
    k=2  # returns top 3 most relevant articles
)

[(Document(page_content="Title: 'Picoflare jets power the solar wind emerging from a coronal hole on the Sun', Abstract: Coronal holes are areas on the Sun with open magnetic field lines. They are a source region of the solar wind, but how the wind emerges from coronal holes is not known. We observed a coronal hole using the Extreme Ultraviolet Imager on the Solar Orbiter spacecraft. We identified jets on scales of a few", metadata={}),
  0.25404146),
 (Document(page_content='hole. We suggest that such picoflare jets could produce enough high-temperature plasma to sustain the solar wind and that the wind emerges from coronal holes as a highly intermittent outflow at small scales.', metadata={}),
  0.25763255)]

In [ ]:
llm('How are solar winds connected to picoflare jets')

"?\n\nSolar winds and picoflare jets are both high-energy phenomena that occur in the vicinity of the Sun. While they are distinct events, there is some evidence to suggest that they may be connected. Here are some possible ways in which solar winds and picoflare jets might be related:\n\n1. Magnetic reconnection: Both solar winds and picoflare jets involve the release of magnetic energy, which can be triggered by magnetic reconnection events. During a magnetic reconnection event, the magnetic field lines in the solar atmosphere break and reconnect, releasing a large amount of energy in the form of heat and radiation. This energy can then drive the acceleration of charged particles, such as electrons and ions, which can produce the observed effects of solar winds and picoflare jets.\n2. Coronal heating: The corona, the outermost layer of the Sun's atmosphere, is heated to millions of degrees Celsius during a flare or other energetic event. This heating can cause the plasma in the coron

In [ ]:
rag('How are solar winds connected to picoflare jets?')



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'How are solar winds connected to picoflare jets?',
 'result': ' Picoflare jets are powered by magnetic reconnection and have kinetic energy in the picoflare range. They are intermittent but widespread within the observed coronal hole.'}

RAG gives concrete answer based on recent paper whereas LLM mostly rambles on.

Summary:

Vector store component of RAG system seems to perform well, especially when chunks of text are not too big (400 tokens). The answers produced by RAG seem slightly more succinct than with pure Llama which uses its parametric knowledge to enrich the answer.